## Solo Exploration and Presentation:

available on: https://github.com/nss-data-science-cohort-6/un_data_question-tomoumer

1. Choose and download another data set from the UN data http://data.un.org/Explorer.aspx to explore. You may want to combine your new dataset with one or both of the datasets that you already worked with. Prepare a short (< 5 minute) presentation of your findings. Report any interesting correlations or trends that you find.

2. If time allows, check out the plotly library to add additional interativity to your plots. https://plotly.com/python/plotly-express/.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [3]:
gdp_df = (
    pd.read_csv('../data/gdp_per_capita.csv', skipfooter=2, engine='python')
    .drop(columns='Value Footnotes')
    .rename(columns={'Country or Area': 'Country', 'Value': 'GDP_Per_Capita'})
)

In [4]:
trade = (
    pd.read_csv('../data/tradeofgoods.csv')
    .rename(columns = {'Country or Area' : 'Country'})
    .drop(columns = ['Commodity', 'Weight (kg)', 'Quantity Name', 'Quantity'])
    )

trade['Country'] = (trade['Country']
    .str.replace('USA', 'United States')
    .str.replace('Brunei Darussalam', 'Brunei')
    .str.replace(r'Bolivia.+', 'Bolivia', regex=True)
)
# note: had to use regex to rename Bolivia, because the parenthesis messed up the string Bolivia (Plurinational State of)
# trade['Country'].unique() to check them

In [7]:
gdp_trade = pd.merge(gdp_df, trade, "inner", on=['Country', 'Year'])

I wanted to check a graph with some countries, decided to go with: gdp_trade.loc[gdp_trade['Country'].isin(['Slovenia', 'Italy', 'United States', 'China'])]

In [17]:
country_filter = ['Slovenia', 'Italy', 'United States', 'China']
plots_per_row = 2
plot_x = 'Year'
plot_y = 'Trade (USD)'
hue_var = 'Flow'

In [22]:
def multicountry_plot(country_filter, plots_per_row, plot_x, plot_y, hue_var='') :
    g1 = (sns.FacetGrid
        (gdp_trade
        .loc[gdp_trade['Country']
        .isin(country_filter)],
        col='Country',
        hue=hue_var,
        height=6,
        col_wrap=plots_per_row
        )
        .map(sns.scatterplot,
        plot_x,
        plot_y
        )
        .add_legend()
        );

In [ ]:
# I made plotting into a function!! Yay!

Comment on above graphs: Slo's trade is ofc negligible compared to US. What's interesting though is that Italy seems to be oscillating around the same value, while the US is consistently on the rise, though the imports are outpacing the exports. China instead is exporting more than importing.

Idea: look at top 10 countries with highest GDP and check different statistics for them

In [ ]:
gdp_df.head()

In [ ]:
(gdp_df
    .loc[gdp_df['Year'] == gdp_df['Year'].max()]
    .sort_values('GDP_Per_Capita', ascending=False)
    .head(11)
)

I selected top 11 countries for the last year available (2019) sorted from highest GDP_Per_Capita to lowest. The reason I did top 11 is because Macao SAR, China is technically a self-governing(*ish) City, so I haven't decided if I'm going to include it or exclude for further comparisons. Below the top 10 for comparison.

In [ ]:
(gdp_df
    .loc[gdp_df['Year'] == gdp_df['Year'].max()]
    .sort_values('GDP_Per_Capita', ascending=False)
    .tail(10)
)

In [ ]:
top10_gdp = (gdp_df
    .loc[
        (gdp_df['Year'] == gdp_df['Year'].max()) &
        (gdp_df['Country'] != 'Macao SAR, China')
        ]
    .sort_values('GDP_Per_Capita', ascending=False)
    .head(10)['Country']
)

In [ ]:
g2 = sns.FacetGrid(
    gdp_trade.loc[gdp_trade['Country'].isin(top10_gdp)],
    col='Country',
    col_wrap=4,
    hue='Flow',
    height=6
)
g2.map(sns.scatterplot, 'Year', 'Trade (USD)')
g2.add_legend()

Interestingly enough, the other 9 top countries per GDP per capita can't compare to the UDS trade. I'll have to normalize the trade values per capita.

In [ ]:
population = pd.read_csv('../data/Population1990to2019.csv')

This data has already been filtered from 1990 to 2019, because the database was too big to download. Furthermore, this data includes predictors for future years; for past years it's all the same, so I decided to keep only the 'no change' (although it doesn't matter)

In [ ]:
population = (
    population
    .loc[population['Variant'] == 'No change']
    .drop(columns='Variant')
    .rename(columns={'Country or Area' : 'Country', 'Year(s)' : 'Year', 'Value' : 'Population'})
)
population['Country'] = population['Country'].str.replace('United States of America', 'United States')
population['Country'] = population['Country'].str.replace('Brunei Darussalam', 'Brunei')
population['Country'] = population['Country'].str.replace(r'Bolivia.+', 'Bolivia', regex=True)

In [ ]:
#did not realize this at first, the population was in thousands; need to multiply!
population['Population'] = population['Population'] * 1000

In [ ]:
gdp_trade_pop = pd.merge(gdp_trade, population, "inner", on=['Country', 'Year'])

Adjusting the trade to trade per capita (dividing Trade (USD) by Population)

In [ ]:
gdp_trade_pop['Trade_Per_Capita'] = gdp_trade_pop['Trade (USD)'] / gdp_trade_pop['Population'] 

In [ ]:
gdp_trade_pop.head()

In [ ]:
g3 = sns.FacetGrid(
    gdp_trade_pop.loc[gdp_trade_pop['Country'].isin(top10_gdp)],
    col='Country',
    col_wrap=4,
    hue='Flow',
    height=6
)
g3.map(sns.scatterplot, 'Year', 'Trade_Per_Capita')
g3.add_legend()

I had to go back and fix the USA / United States / United States of America, as well as Brunei Darussalam to Brunei

When adjusted for population, the graphs of trade paint quite a different picture. In terms of exporting goods, Singapore, Switzerland and Arab Emirates are outstanding. Also Ireland - that's the one I'm most surprised about in all honesty.

In [ ]:
g4 = sns.FacetGrid(
    gdp_trade_pop.loc[gdp_trade_pop['Country'].isin(top10_gdp)],
    col='Country',
    col_wrap=5
)
g4.map(sns.scatterplot, 'Year', 'GDP_Per_Capita')
g4.add_legend()

In [ ]:
g5 = sns.FacetGrid(
    gdp_trade_pop.loc[gdp_trade_pop['Country'].isin(top10_gdp)],
    col='Country',
    col_wrap=4,
    hue='Flow',
    height=5
)
g5.map(sns.scatterplot, 'Trade_Per_Capita', 'GDP_Per_Capita')
g5.add_legend()

Above is the plot I was initially curious to see. comparing hte GDP Per capita to the Trade Per Capita. I was wondering what impact does the trade have on GDP. Without any research I'd imagine that if we're looking just at Import/Export, the more the difference between those two would be, the higher the GDP. Interestingly, I don't see such a clear trend for most countries.

In [ ]:
gdp_trade_pop.head()

In [ ]:
# don't do for loops in pandas ... if you do, typically means you did something wrong
# if (row['Flow'] == 'Import') or (row['Flow'] == 'Re-Import'):
#    gdp_trade_pop.loc[index, 'Trade (USD)'] = gdp_trade_pop.loc[index, 'Trade (USD)'] * (-1)
#    gdp_trade_pop.loc[index, 'Trade_Per_Capita'] = gdp_trade_pop.loc[index, 'Trade_Per_Capita'] * (-1)

In [ ]:
# same as pivot below
# gdp_trade_pop.pivot_table(index=['Country', 'Year', 'GDP_Per_Capita', 'Population'], columns=['Flow']).reset_index()

In [ ]:
# this below would work, except I'd need to find a way to get rid of the sub-columns
gdp_trade_pop.pivot(index=['Country', 'Year', 'GDP_Per_Capita', 'Population'], columns='Flow').reset_index()

In [ ]:
# the function below is made in order to separate the Trade(USD);
# only downside is that with this method, Trade_Per_Capita could not be kept so I'll need to calculate it again
gdp_trade_pop = (
    gdp_trade_pop
    .pivot(index=['Country', 'Year', 'GDP_Per_Capita', 'Population'],
        columns='Flow',
        values='Trade (USD)')
    .reset_index()
    .rename_axis(None, axis=1)
    .fillna(0)
)

In [ ]:
gdp_trade_pop['Surplus_Deficit'] = gdp_trade_pop['Export'] + gdp_trade_pop['Re-Export'] - gdp_trade_pop['Import'] - gdp_trade_pop['Re-Import']
gdp_trade_pop['Export_Per_Capita'] = gdp_trade_pop['Export'] / gdp_trade_pop['Population'] 
gdp_trade_pop['Import_Per_Capita'] = gdp_trade_pop['Import'] / gdp_trade_pop['Population'] 
gdp_trade_pop['Re-Export_Per_Capita'] = gdp_trade_pop['Re-Export'] / gdp_trade_pop['Population'] 
gdp_trade_pop['Re-Import_Per_Capita'] = gdp_trade_pop['Re-Import'] / gdp_trade_pop['Population']
gdp_trade_pop['Surplus_Deficit_Per_Capita'] = gdp_trade_pop['Surplus_Deficit'] / gdp_trade_pop['Population']

Now that I have this, let's look at some countries (irrespective of year) who had the biggest surplus, or deficit based on their exports and imports.

I'm surprised that for many years Luxembourg appears amongst the worst!

In [ ]:
gdp_trade_pop.sort_values(by='Surplus_Deficit_Per_Capita').head(10)

In [ ]:
gdp_trade_pop.sort_values(by='Surplus_Deficit_Per_Capita').tail(10)

Just checking now the same, but only for the year 2019

In [ ]:
gdp_trade_pop.loc[gdp_trade_pop['Year'] == gdp_trade_pop['Year'].max()].sort_values(by='Surplus_Deficit_Per_Capita').head(10)


In [ ]:
gdp_trade_pop.loc[gdp_trade_pop['Year'] == gdp_trade_pop['Year'].max()].sort_values(by='Surplus_Deficit_Per_Capita').tail(10)


Luxemburg disappeared from bottom 10 in 2019, yay! With that said, of the top 10 GDP earners, only Switzerland appears to have the highest surplus! Also, alright Poland, those Witcher sales looking good for your economy, lol!

In [ ]:
g6 = sns.FacetGrid(
    gdp_trade_pop.loc[gdp_trade_pop['Country'].isin(top10_gdp)],
    col='Country',
    col_wrap=4,
    hue='Year',
    height=6
)
g6.map(sns.scatterplot, 'Surplus_Deficit_Per_Capita', 'GDP_Per_Capita')
g6.add_legend()

A more clear look at what above - a higher surplus does not necessarily mean higher GDP.

In [ ]:
sns.scatterplot(
    data=gdp_trade_pop.loc[gdp_trade_pop['Country']=='Slovenia'],
    x='Surplus_Deficit_Per_Capita',
    y='GDP_Per_Capita',
    hue='Year'
)

In [ ]:
sns.lineplot(
    data=gdp_trade_pop.loc[gdp_trade_pop['Country']=='Slovenia'],
    x='Year',
    y='GDP_Per_Capita'
)


In [ ]:
sns.lineplot(
    data=gdp_trade_pop.loc[gdp_trade_pop['Country']=='Slovenia'],
    x='Year',
    y='Surplus_Deficit_Per_Capita'
)

After all this: idea to compare the highest GDP and lowest GDP per different continents.

In [ ]:
continents = pd.read_csv('../data/continents.csv')

In [ ]:
gdp_trade_pop_continent = pd.merge(gdp_trade_pop, continents, how='inner', on='Country')

In [ ]:
gdp_trade_pop_continent.value_counts('Continent')

First of all, create a list of top GDP countries per each continent and bottom GDP countries per each continent (6 in each list).

In [ ]:
gdp_trade_pop_continent.groupby('Continent')['GDP_Per_Capita'].describe()

Comment: I knew that they were going to be skewed (with some countries having much higher gdp than others), but something doesn't look right!

In [ ]:
gdp_trade_pop_continent.loc[gdp_trade_pop_continent['Year'] == gdp_trade_pop_continent['Year'].max()].groupby('Continent')['GDP_Per_Capita'].describe()

In [ ]:
gdp_trade_pop_continent.loc[gdp_trade_pop_continent['Year']==2019]

## After all this, just realized that trade might not have all the values I need for year 2019 because of various INNER merges!

Ok, back to the original gdp_df file, to merge it with continents and see what's up.

In [ ]:
gdp_continent = pd.merge(gdp_df, continents, how='inner', on='Country')

In [ ]:
gdp_continent.loc[gdp_continent['Year'] == 2019].groupby('Continent')['GDP_Per_Capita'].describe()

In [ ]:
max_gdp2019 = gdp_continent.loc[(gdp_continent['Year'] == 2019) & (gdp_continent['Country'] != 'Macao SAR, China')].groupby('Continent')['GDP_Per_Capita'].max()
max_gdp2019_countries = gdp_continent.loc[gdp_continent['GDP_Per_Capita'].isin(max_gdp2019)]['Country']

In [ ]:
max_gdp2019_countries

After checking for countries in the Trade dataset, I realized that most of these don't have a value for 2019 (last one is 2018). And Macao SAR, China is weird, stops at 2016, so excluding that one anyways!

After I ridid the same exact as above, except 2018, the same top 6 countries appeared!

can check the years with something like this trade.loc[trade['Country'] == 'Ukraine'].sort_values('Year', ascending=False)

Haiti only has values until 1997..., Honduraas 2017. For year 2019 Nicaragua has lower gdp than Honduras, and it does go until 2018, so I went with that. Also had to remove Kosovo as it's not in the Trade dataset!

In [ ]:
min_gdp2019 = gdp_continent.loc[(gdp_continent['Year'] == 2019) & (gdp_continent['Country'] != 'Haiti') & (gdp_continent['Country'] != 'Kosovo')].groupby('Continent')['GDP_Per_Capita'].min()
min_gdp2019_countries = gdp_continent.loc[gdp_continent['GDP_Per_Capita'].isin(min_gdp2019)]['Country']

In [ ]:
min_gdp2019_countries

In [ ]:
minmax_gdp2019_countries = pd.concat([min_gdp2019_countries, max_gdp2019_countries])

In [ ]:
minmax_gdp2019_countries

In [ ]:
# by using code below, I found out which country was missing and it was Bolivia!
population.loc[population['Country'].isin(minmax_gdp2019_countries)].value_counts('Country')

In [ ]:
filtered_gdp_cont = gdp_continent.loc[gdp_continent['Country'].isin(minmax_gdp2019_countries)]

In [ ]:
filtered_gdp_cont_pop = pd.merge(filtered_gdp_cont, population, "inner", on=['Country', 'Year'])

In [ ]:
filtered_gdp_cont_pop.info()

I want to keep all the years from GDP, so I'll have to left join because trade is missing years

In [ ]:
filtered_gdp_cont_pop_trade = pd.merge(filtered_gdp_cont_pop, trade, 'left', on=['Country', 'Year'])

In [ ]:
filtered_gdp_cont_pop_trade = (
    filtered_gdp_cont_pop_trade
    .pivot(index=['Country', 'Year', 'GDP_Per_Capita', 'Population'],
        columns='Flow',
        values='Trade (USD)')
    .reset_index()
    .rename_axis(None, axis=1)
    .fillna(0)
)
# uh so this is weird, by doing a left join instead of inner, this leaves me with a NaN column that
# doesn't want to be dropped using .drop(columns='NaN')

In [ ]:
filtered_gdp_cont_pop_trade['Surplus_Deficit'] = filtered_gdp_cont_pop_trade['Export'] + filtered_gdp_cont_pop_trade['Re-Export'] - filtered_gdp_cont_pop_trade['Import'] - filtered_gdp_cont_pop_trade['Re-Import']
filtered_gdp_cont_pop_trade['Export_Per_Capita'] = filtered_gdp_cont_pop_trade['Export'] / filtered_gdp_cont_pop_trade['Population'] 
filtered_gdp_cont_pop_trade['Import_Per_Capita'] = filtered_gdp_cont_pop_trade['Import'] / filtered_gdp_cont_pop_trade['Population'] 
filtered_gdp_cont_pop_trade['Re-Export_Per_Capita'] = filtered_gdp_cont_pop_trade['Re-Export'] / filtered_gdp_cont_pop_trade['Population'] 
filtered_gdp_cont_pop_trade['Re-Import_Per_Capita'] = filtered_gdp_cont_pop_trade['Re-Import'] / filtered_gdp_cont_pop_trade['Population']
filtered_gdp_cont_pop_trade['Surplus_Deficit_Per_Capita'] = filtered_gdp_cont_pop_trade['Surplus_Deficit'] / filtered_gdp_cont_pop_trade['Population']

In [ ]:
filtered_gdp_cont_pop_trade.head(5)

All is ready for the seaborn plots!!

In [ ]:
g7 = sns.FacetGrid(
    filtered_gdp_cont_pop_trade,
    col='Country',
    col_wrap=4,
    hue='Year'
)
g7.map(sns.scatterplot, 'Surplus_Deficit_Per_Capita', 'GDP_Per_Capita')
g7.add_legend()

In [ ]:
g7max = sns.FacetGrid(
    filtered_gdp_cont_pop_trade.loc[filtered_gdp_cont_pop_trade['Country'].isin(max_gdp2019_countries)],
    col='Country',
    col_wrap=3,
    hue='Year',
    height=6
)
g7max.map(sns.scatterplot, 'Surplus_Deficit_Per_Capita', 'GDP_Per_Capita')
g7max.add_legend()

In [ ]:
g7min = sns.FacetGrid(
    filtered_gdp_cont_pop_trade.loc[filtered_gdp_cont_pop_trade['Country'].isin(min_gdp2019_countries)],
    col='Country',
    col_wrap=3,
    hue='Year',
    height=6
)
g7min.map(sns.scatterplot, 'Surplus_Deficit_Per_Capita', 'GDP_Per_Capita')
g7min.add_legend()

In [ ]:
g8 = sns.FacetGrid(
    filtered_gdp_cont_pop_trade,
    col='Country',
    col_wrap=4
)
g8.map(sns.scatterplot, 'Year', 'GDP_Per_Capita')
g8.add_legend()

In [ ]:
g9 = sns.FacetGrid(
    filtered_gdp_cont_pop_trade,
    col='Country',
    col_wrap=4
)
g9.map(sns.scatterplot, 'Year', 'Surplus_Deficit_Per_Capita')
g9.add_legend()

In [ ]:
fig1 = px.line(filtered_gdp_cont_pop_trade.loc[filtered_gdp_cont_pop_trade['Country'].isin(max_gdp2019_countries)], x='Year', y='GDP_Per_Capita', color='Country', symbol="Country")
fig1.show()

In [ ]:
fig2 = px.line(filtered_gdp_cont_pop_trade.loc[filtered_gdp_cont_pop_trade['Country'].isin(min_gdp2019_countries)], x='Year', y='GDP_Per_Capita', color='Country', symbol="Country")
fig2.show()

In [ ]:
fig3 = px.line(filtered_gdp_cont_pop_trade.loc[filtered_gdp_cont_pop_trade['Country'].isin(max_gdp2019_countries)], x='Year', y='Surplus_Deficit_Per_Capita', color='Country', symbol="Country")
fig3.show()

In [ ]:
fig4 = px.line(filtered_gdp_cont_pop_trade.loc[filtered_gdp_cont_pop_trade['Country'].isin(min_gdp2019_countries)], x='Year', y='Surplus_Deficit_Per_Capita', color='Country', symbol="Country")
fig4.show()

Looking at the two graphs above: it is interesting how for the top GDP countries, the surplus (or lack thereoff) oscilates in larger values - up to 20 Million in USD. For poorest GDP countries it is contained within a Million

In [ ]:
fig5 = px.scatter(filtered_gdp_cont_pop_trade.loc[filtered_gdp_cont_pop_trade['Country'] == 'Luxembourg'], x='Surplus_Deficit_Per_Capita', y='GDP_Per_Capita', color='Year')
fig5.show()

alright, let's do the above in a function!

In [ ]:
def gd